<a href="https://colab.research.google.com/github/MeikisPlay/image_processing/blob/main/14_11_24_text_generation_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание: Генерация стихов с использованием статистической языковой модели
Цель задания: Научиться генерировать стихотворные тексты, используя марковскую цепь и статистическую языковую модель. Разработать код на Python для генерации стихов с применением вероятностных методов.

## 1. Определения
В этом задании вы будете использовать марковские цепи для генерации стихотворных строк. Мы будем работать с простым текстом стихов и создавать модель, которая будет предсказывать следующее слово на основе предыдущего

Марковские цепи — это математическая модель, описывающая систему, которая изменяется со временем, переходя из одного состояния в другое с некоторой вероятностью. Основная особенность марковских цепей в том, что будущее состояние зависит только от текущего состояния и не зависит от того, как система попала в это состояние. Это свойство называется марковским свойством или свойством отсутствия памяти. Используется в самых разных областях, начиная генерацией текста и заканчивая финансовым моделированием.

### Пример
Допустим, у нас есть текст: "кот гуляет по двору. кот ловит мышь. мышь убегает от кота."

Для марковской цепи, построенной по одному слову, вероятности переходов могут быть такими:

После слова "кот" с вероятностью 0.5 идет "гуляет" и с вероятностью 0.5 идет "ловит".
После слова "гуляет" идет "по".
После слова "по" идет "двору".
...
Марковская цепь может быть использована для генерации новых предложений, основанных на вероятностях переходов, что позволит создать фразы, похожие по структуре на исходный текст, но уникальные по содержанию.

## 2. Подготовка данных
Для начала, вам нужно выбрать корпус стихотворных текстов. Это может быть как один поэт, так и несколько авторов. Например, можно использовать стихи Пушкина или Лермонтова.

Загрузите текст в Python, обработайте его и подготовьте для обучения языковой модели.



In [2]:
import re

# Загрузка текста из файла
def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Очистка текста (удаление знаков препинания и переводов строк)
def clean_text(text):
    # Удаляем знаки препинания и переводы строк, заменяем несколько пробелов на один
    text = re.sub(r'[^\w\s]', '', text)  # Удаляем все, кроме букв и пробелов
    text = re.sub(r'\s+', ' ', text)     # Заменяем несколько пробелов на один
    return text.strip()                  # Удаляем пробелы в начале и конце

# Пример использования
text = load_text('/content/ЕвгенийОнегин.txt')  # Замените на свой файл с текстами
cleaned_text = clean_text(text)
print(cleaned_text[:500])  # Выведем первые 500 символов

Не мысля гордый свет забавить Вниманье дружбы возлюбя Хотел бы я тебе представить Залог достойнее тебя Достойнее души прекрасной Святой исполненной мечты Поэзии живой и ясной Высоких дум и простоты Но так и быть рукой пристрастной Прими собранье пестрых глав Полусмешных полупечальных Простонародных идеальных Небрежный плод моих забав Бессониц легких вдохновений Незрелых и увядших лет Ума холодных наблюдений И сердца горестных замет И жить торопится и чувствовать спешит Кн Вяземский Мой дядя самы


## 3. Создание модели на основе марковской цепи
Теперь, когда у нас есть очищенный текст, мы можем создать модель марковской цепи, которая будет предсказывать следующее слово на основе предыдущего. Нужно составить словарь, в котором для каждого слова будет указан список слов, которые встречаются в тексте после него.

Например:

Пример марковской цепи: ('поп', ['толоконный', 'по', 'ему', 'стал', 'говорит', 'один', 'ни', 'завидя', 'подставил', 'до', 'языка', 'за', 'одной', 'замолчал', 'он', 'морочил'])
то есть после слова 'поп' в тексте встречаются слова: 'толоконный', 'по', 'ему', 'стал', 'говорит', 'один', 'ни', 'завидя', 'подставил', 'до', 'языка', 'за', 'одной', 'замолчал', 'он', 'морочил'

Еще примеры ниже:

Пример марковской цепи: ('год', ['за', 'прошел', 'по', 'другой', 'святой', 'провел', 'иль', 'в', 'один', 'летит', 'ему', 'проходит', 'я', 'и', 'и', 'о', 'с', 'опять', 'проходит'])


Пример марковской цепи: ('него', ['пляшет', 'заране', 'прикрикнула', 'старуха', 'она', 'за', 'за', 'летал', 'сатиров', 'найдем', 'ты', 'не', 'со', 'правдиву', 'друзья', 'невидимым', 'резец', 'певец', 'прямым', 'и', 'уж', 'тиран', 'я', 'не', 'наскочили', 'он', 'марко', 'злые', 'под', 'моих', 'наконец', 'перед', 'попрежнему', 'взвился'])


Пример марковской цепи: ('лоб', ['пошел', 'щелк', 'у', 'от', 'с', 'с', 'рукою', 'широк', 'шлагбаум', 'левее', 'ему'])


Пример марковской цепи: ('череда', ['условия'])


In [7]:
import re
from collections import defaultdict


def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


def clean_text(text):

    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()


def build_markov_chain(text):
    words = text.split()
    markov_chain = defaultdict(list)

    for i in range(len(words) - 1):
        markov_chain[words[i]].append(words[i + 1])

    return markov_chain


text = load_text('ЕвгенийОнегин.txt')
cleaned_text = clean_text(text)
markov_chain = build_markov_chain(cleaned_text)


word_to_check = 'жилбыл'
if word_to_check in markov_chain:
    print(f"{word_to_check}: {markov_chain[word_to_check]}")
else:
    print(f"Слово '{word_to_check}' не найдено в марковской цепи.")

Слово 'жилбыл' не найдено в марковской цепи.


In [9]:
markov_chain['пошел']

['пошел', 'андрюшка', 'ведет', 'уже']

## 4. Генерация стихотворных строк
Теперь, используя созданную модель, мы будем генерировать строки стихов. Начнем с случайного выбора стартового слова и будем добавлять следующее слово на основе модели.

In [11]:
import re
import random
from collections import defaultdict

def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()

def build_markov_chain(text):
    words = text.split()
    markov_chain = defaultdict(list)

    for i in range(len(words) - 1):
        markov_chain[words[i]].append(words[i + 1])

    return markov_chain

text = load_text('ЕвгенийОнегин.txt')
cleaned_text = clean_text(text)
markov_chain = build_markov_chain(cleaned_text)

words = ['пошел', 'андрюшка', 'ведет', 'уже']

def generate_text(model, words, n=2, length=10):
    start_gram = random.choice(words)
    print("Начальное слово:", start_gram)
    output = [start_gram]

    for _ in range(length - n + 1):
        next_word = random.choice(model.get(start_gram, ['']))
        if not next_word:
            break
        output.append(next_word)
        start_gram = next_word

    return ' '.join(output)

generated_poem = generate_text(markov_chain, words, n=2, length=20)

print(f"Сгенерированное стихотворение: \n{generated_poem}")

Начальное слово: пошел
Сгенерированное стихотворение: 
пошел ведет так бледно равнодушно они поют и опершися на стекла хладные мечты текут элегии венок убогой и каждый взял


5. Генерация стихотворных строк с рифмой
Для более поэтичного результата можно попробовать добавить рифму в процесс генерации. Это будет сложнее, но возможно с использованием специальных алгоритмов, например, для поиска рифм.

### 6. Анализ результатов
Проанализируйте сгенерированные строки:

    Строки рифмуются ли между собой?
    Есть ли логичность в генерируемом тексте?



####  Дополнительное задание (необязательно):
Оцените, как улучшить модель:
    Попробуйте создать более сложную модель, учитывая биграммы или триграммы. Это может улучшить структуру и ритм.
    Попробуйте экспериментировать с другими корпусами текстов (например, смените поэта).
    Модифицируйте модель, чтобы она могла генерировать стихи с определенной тематикой (например, о природе или любви)